In [12]:
import sys
!{sys.executable} -m pip install sodapy

import pandas as pd
from sodapy import Socrata
import numpy as np

#retrieving Food Inspections data using Socrata and converting to DF
client = Socrata("data.cityofchicago.org", None)
results = client.get("cwig-ma7x", limit=2000) #limit set for ease of processing use
inspections_df = pd.DataFrame.from_records(results)

inspections_df.head(n=10)

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_vrxf_vc4k,address,aka_name,city,dba_name,facility_type,...,inspection_type,latitude,license_,location,longitude,results,risk,state,violations,zip
0,16,4451,40,539,76,5657-5659 N CLARK ST,ODA RESTAURANT,CHICAGO,ODA RESTAURANT LLC,Restaurant,...,License Re-Inspection,41.98497216927604,2658415,"{'type': 'Point', 'coordinates': [-87.66892851...",-87.66892851302207,Pass,Risk 1 (High),IL,39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,60660
1,38,21869,28,751,14,4821 N ELSTON AVE,SENOR PAN CAFE,CHICAGO,SENOR PAN CAFE,Restaurant,...,License,41.96856245387149,2658933,"{'type': 'Point', 'coordinates': [-87.74065931...",-87.74065931463221,Fail,Risk 1 (High),IL,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",60630
2,28,21569,14,200,32,3120-3126 S KOSTNER,LITTLE VILLAGE HIGH SCHOOL,CHICAGO,LITTLE VILLAGE HIGH SCHOOL,School,...,Canvass,41.836019534753326,49121,"{'type': 'Point', 'coordinates': [-87.73419465...",-87.73419465301697,Fail,Risk 1 (High),IL,"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",60623
3,42,4447,33,256,73,3043-3047 W 111TH ST,BARRACO'S PIZZA,CHICAGO,BARRACO'S PIZZA,Restaurant,...,Canvass,41.691495324148114,2303698,"{'type': 'Point', 'coordinates': [-87.69778040...",-87.6977804013201,Fail,Risk 1 (High),IL,2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,60655
4,15,22268,23,265,62,5904 1/2 W 63RD ST,LA MICHOACANNA PREFERIDA,CHICAGO,LA MICHOACANNA PREFERIDA,Restaurant,...,Canvass,41.778012898008846,2099471,"{'type': 'Point', 'coordinates': [-87.76930105...",-87.7693010599763,Pass w/ Conditions,Risk 2 (Medium),IL,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,60638
5,48,14917,8,97,29,727 W MAXWELL ST,THE MOMO WORLD,CHICAGO,THE MOMO WORLD,Restaurant,...,License Re-Inspection,41.86468356389474,2641613,"{'type': 'Point', 'coordinates': [-87.64613291...",-87.64613291942946,Pass,Risk 1 (High),IL,57. ALL FOOD EMPLOYEES HAVE FOOD HANDLER TRAIN...,60607
6,8,22268,35,607,53,5842 S ARCHER AVE,CAFE ROSA,CHICAGO,CAFE ROSA,Restaurant,...,Canvass,41.79613354689054,2386048,"{'type': 'Point', 'coordinates': [-87.76038586...",-87.7603858621191,Fail,Risk 1 (High),IL,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",60638
7,19,22254,25,26,15,5700-5702 W IRVING PARK RD,VERANDA,CHICAGO,VERANDA GREEK AMERICAN TAVERNA,Restaurant,...,Canvass,41.95325816953041,1445335,"{'type': 'Point', 'coordinates': [-87.76951202...",-87.76951202741661,Out of Business,Risk 1 (High),IL,NaN,60634
8,42,4447,33,256,73,3124 W 111TH ST,BARRACO'S PIZZA,CHICAGO,BARRACO'S PIZZA,BANQUET,...,License,41.691692868643685,2657796,"{'type': 'Point', 'coordinates': [-87.69946849...",-87.69946849520649,Fail,Risk 3 (Low),IL,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,60655
9,21,22535,27,71,22,3924 W DIVERSEY AVE,LA CATRINA RESTAURANT,CHICAGO,LA CATRINA RESTAURANT LLC,Restaurant,...,License Re-Inspection,41.931917684341876,2658196,"{'type': 'Point', 'coordinates': [-87.72554475...",-87.725544754742,Pass w/ Conditions,Risk 1 (High),IL,"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",60647


In [ ]:
#Iterate through to read the full dataset
initial = 2000
while results:
    results = client.get("cwig-ma7x", limit=2000, offset=initial)
    inspections_df = inspections_df.append(pd.DataFrame.from_records(results))
    initial += 2000

inspections_df.count()

In [13]:
#drop uneccesary computed columns 
inspections_df = inspections_df.drop(columns=[':@computed_region_43wa_7qmu', ':@computed_region_6mkv_f3dw', ':@computed_region_awaf_s7ux'
                            ,':@computed_region_bdys_3d7i', ':@computed_region_vrxf_vc4k', 'location', 'city', 'state', 'aka_name'])
inspections_df.head()

,address,dba_name,facility_type,inspection_date,inspection_id,inspection_type,latitude,license_,longitude,results,risk,violations,zip
0,5657-5659 N CLARK ST,ODA RESTAURANT LLC,Restaurant,2019-04-10T00:00:00.000,2282922,License Re-Inspection,41.98497216927604,2658415,-87.66892851302207,Pass,Risk 1 (High),39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,60660
1,4821 N ELSTON AVE,SENOR PAN CAFE,Restaurant,2019-04-10T00:00:00.000,2282916,License,41.96856245387149,2658933,-87.74065931463221,Fail,Risk 1 (High),"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",60630
2,3120-3126 S KOSTNER,LITTLE VILLAGE HIGH SCHOOL,School,2019-04-10T00:00:00.000,2282913,Canvass,41.836019534753326,49121,-87.73419465301697,Fail,Risk 1 (High),"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",60623
3,3043-3047 W 111TH ST,BARRACO'S PIZZA,Restaurant,2019-04-09T00:00:00.000,2282879,Canvass,41.691495324148114,2303698,-87.6977804013201,Fail,Risk 1 (High),2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,60655
4,5904 1/2 W 63RD ST,LA MICHOACANNA PREFERIDA,Restaurant,2019-04-09T00:00:00.000,2282882,Canvass,41.778012898008846,2099471,-87.7693010599763,Pass w/ Conditions,Risk 2 (Medium),5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,60638


In [14]:
#Drop rows with null data
inspections_df.dropna()
inspections_df.count()

address            2000
dba_name           2000
facility_type      1979
inspection_date    2000
inspection_id      2000
inspection_type    2000
latitude           1990
license_           2000
longitude          1990
results            2000
risk               1999
violations         1540
zip                1996
dtype: int64

In [15]:
#Checking whether there are any duplicates
#If duplicates exist, then use .drop_duplicates()
d = inspections_df.duplicated()

duplicates_amount = 0 
duplicates = []
try:
    for i in range(1,len(d)):
        if d[i] == True:
            duplicates_amount = duplicates_amount + 1
            duplicates.append(i)
            print("Duplicates Amount: ",duplicates_amount) 
except:
    print("No duplicates in this dataset")

In [16]:
#Remove trailing characters from date field
inspections_df["inspection_date"] = inspections_df.inspection_date.str.split('T').str[0]
inspections_df.inspection_date.head(n=10)

0    2019-04-10
1    2019-04-10
2    2019-04-10
3    2019-04-09
4    2019-04-09
5    2019-04-09
6    2019-04-09
7    2019-04-09
8    2019-04-09
9    2019-04-09
Name: inspection_date, dtype: object

In [17]:
#Exclude unsuccessful inspections
inspections_df = inspections_df[~inspections_df.results.isin(["Business Not Located", "Out of Business"])] 
inspections_df.count()

address            1898
dba_name           1898
facility_type      1885
inspection_date    1898
inspection_id      1898
inspection_type    1898
latitude           1889
license_           1898
longitude          1889
results            1898
risk               1897
violations         1539
zip                1894
dtype: int64

In [18]:
#Format data types - can use as_type() to convert 
inspections_df.dtypes

address            object
dba_name           object
facility_type      object
inspection_date    object
inspection_id      object
inspection_type    object
latitude           object
license_           object
longitude          object
results            object
risk               object
violations         object
zip                object
dtype: object

In [19]:
#Rename columns due to sodapy conversion
inspections_df = inspections_df.rename(columns={"license_":"License", "address":"Address",
                               "dba_name":"DBA Name", "facility_type":"Facility Type", "inspection_date":"Inspection Date",
                              "inspection_id":"Inspection ID", "inspection_type":"Inspection Type", "latitude":"Latitude",
                              "longitude":"Longitude", "results":"Results", "risk":"Risk", 
                              "violations":"Violations", "zip":"Zip"})
inspections_df.head()

,Address,DBA Name,Facility Type,Inspection Date,Inspection ID,Inspection Type,Latitude,License,Longitude,Results,Risk,Violations,Zip
0,5657-5659 N CLARK ST,ODA RESTAURANT LLC,Restaurant,2019-04-10,2282922,License Re-Inspection,41.98497216927604,2658415,-87.66892851302207,Pass,Risk 1 (High),39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,60660
1,4821 N ELSTON AVE,SENOR PAN CAFE,Restaurant,2019-04-10,2282916,License,41.96856245387149,2658933,-87.74065931463221,Fail,Risk 1 (High),"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",60630
2,3120-3126 S KOSTNER,LITTLE VILLAGE HIGH SCHOOL,School,2019-04-10,2282913,Canvass,41.836019534753326,49121,-87.73419465301697,Fail,Risk 1 (High),"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",60623
3,3043-3047 W 111TH ST,BARRACO'S PIZZA,Restaurant,2019-04-09,2282879,Canvass,41.691495324148114,2303698,-87.6977804013201,Fail,Risk 1 (High),2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,60655
4,5904 1/2 W 63RD ST,LA MICHOACANNA PREFERIDA,Restaurant,2019-04-09,2282882,Canvass,41.778012898008846,2099471,-87.7693010599763,Pass w/ Conditions,Risk 2 (Medium),5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,60638


In [20]:
#Separate DF into Inspections and Violations
inspections = inspections_df.loc[:, inspections_df.columns != 'Violations']
inspections.head()

,Address,DBA Name,Facility Type,Inspection Date,Inspection ID,Inspection Type,Latitude,License,Longitude,Results,Risk,Zip
0,5657-5659 N CLARK ST,ODA RESTAURANT LLC,Restaurant,2019-04-10,2282922,License Re-Inspection,41.98497216927604,2658415,-87.66892851302207,Pass,Risk 1 (High),60660
1,4821 N ELSTON AVE,SENOR PAN CAFE,Restaurant,2019-04-10,2282916,License,41.96856245387149,2658933,-87.74065931463221,Fail,Risk 1 (High),60630
2,3120-3126 S KOSTNER,LITTLE VILLAGE HIGH SCHOOL,School,2019-04-10,2282913,Canvass,41.836019534753326,49121,-87.73419465301697,Fail,Risk 1 (High),60623
3,3043-3047 W 111TH ST,BARRACO'S PIZZA,Restaurant,2019-04-09,2282879,Canvass,41.691495324148114,2303698,-87.6977804013201,Fail,Risk 1 (High),60655
4,5904 1/2 W 63RD ST,LA MICHOACANNA PREFERIDA,Restaurant,2019-04-09,2282882,Canvass,41.778012898008846,2099471,-87.7693010599763,Pass w/ Conditions,Risk 2 (Medium),60638


In [21]:
violations = inspections_df.loc[:, inspections_df.columns != 'Inspection Date']
violations.head()

,Address,DBA Name,Facility Type,Inspection ID,Inspection Type,Latitude,License,Longitude,Results,Risk,Violations,Zip
0,5657-5659 N CLARK ST,ODA RESTAURANT LLC,Restaurant,2282922,License Re-Inspection,41.98497216927604,2658415,-87.66892851302207,Pass,Risk 1 (High),39. CONTAMINATION PREVENTED DURING FOOD PREPAR...,60660
1,4821 N ELSTON AVE,SENOR PAN CAFE,Restaurant,2282916,License,41.96856245387149,2658933,-87.74065931463221,Fail,Risk 1 (High),"3. MANAGEMENT, FOOD EMPLOYEE AND CONDITIONAL E...",60630
2,3120-3126 S KOSTNER,LITTLE VILLAGE HIGH SCHOOL,School,2282913,Canvass,41.836019534753326,49121,-87.73419465301697,Fail,Risk 1 (High),"38. INSECTS, RODENTS, & ANIMALS NOT PRESENT - ...",60623
3,3043-3047 W 111TH ST,BARRACO'S PIZZA,Restaurant,2282879,Canvass,41.691495324148114,2303698,-87.6977804013201,Fail,Risk 1 (High),2. CITY OF CHICAGO FOOD SERVICE SANITATION CER...,60655
4,5904 1/2 W 63RD ST,LA MICHOACANNA PREFERIDA,Restaurant,2282882,Canvass,41.778012898008846,2099471,-87.7693010599763,Pass w/ Conditions,Risk 2 (Medium),5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,60638


In [22]:
#Create two output tables and third table with entire schema
import os.path
root_path = os.path.dirname(os.getcwd())

#Export to CSV
inspections_df.to_csv(os.path.join(root_path, "Food Inspections Data/Full Food Inspections.csv"), index=False)
inspections.to_csv(os.path.join(root_path, "Food Inspections Data/Inspections.csv"), index=False)
violations.to_csv(os.path.join(root_path, "Food Inspections Data/Violations.csv"), index=False)

In [23]:
#Unit Testing ETL script
import unittest

class TestETL(unittest.TestCase):
    def test_empty_dataframe(self):
        """
        Tests if dataframe is empty
        """
        self.assertIsNotNone(inspections_df)
        
    def test_output_tables(self):
        """
        Tests that the Violations field is not in the inspections table and
        that the Inspection ID field is not in the violations table
        """
        self.assertTrue(inspections.columns.any != 'Violations')
        self.assertTrue(violations.columns.any != "Inspection Date")
        
    def test_filtered_fields(self):
        """
        Tests that unwanted fields get filtered out of DF
        """
        self.assertNotIn("Business Not Located", inspections_df.Results.values)
        self.assertNotIn("Out of Business", inspections_df.Results.values)
        self.assertNotIn(('location', 'city', 'state', 'aka_name'), inspections_df.columns)

unittest.main(argv=[''], verbosity=2, exit=False)

test_empty_dataframe (__main__.TestETL) ... ok
test_filtered_fields (__main__.TestETL) ... ok
test_output_tables (__main__.TestETL) ... ok

----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK
